# Prediction with LSTM model
<p style='text-align: right;'>with selectd1.csv</p>

* history
  * 2025/05/14 PM06:56 : 모델에 대한 기능을 공통화 시킴

In [7]:
def restart_kernel():
    # Restart the kernet after libraries are loaded.
    import IPython
    from datetime import datetime
    print(f'restart kernel... {datetime.now()}')
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [ ]:
!pip install scikit-learn
!pip install tensorflow
from datetime import datetime
print(f'restart kernel... {datetime.now()}')

In [8]:
restart_kernel()

restart kernel... 2025-05-15 17:06:07.360561


In [1]:
# load dependacies
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Input

from globalvar import *
from data import DataLoader
from model1 import PredictLSTM1
from model2 import PredictLSTM2
from model3 import PredictLSTM3
from model4 import PredictLSTM4

import argparse
import matplotlib.pyplot as plt

from util_pred import print_data
from util_pred import flat_data_with_sum, flat_data, get_frequency
from util_pred import save_model, import_mode
from util_pred import get_random_in_list
from util_pred import print_data_with_sort
from util_pred import get_sorted_n_values
from util_pred import dict_key_count
from util_pred import print_list
from util_pred import print_dict_list
from util_pred import print_title
from activation import ActivationOutput, RecurrentActivation
from datetime import datetime

from operate import create_model, create_model_v1, get_predicted

print(f"imported library. ({datetime.now()})")

2025-05-15 17:06:12.978828: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


imported library. (2025-05-15 17:06:14.574672)


In [140]:
##
## 모델링 환경 설정
##
window=1 # default = 1 , help = "time stamps"
data_dir='./selectd2.csv'
    
mode='predict' # help = "back-test or predict")
mode2='sampling' # help = "greed or sampling")
verb='verbose' # help = "verbose or not_verb")
    
trial=20 # help = "how much trials to generate")
training_length=1 # default = 0.9)
epoch=20 # default = 3
batch=1 # default = 1
model_type='lstm4' # help = "lstm1 or lstm2")
hid_dim = 50
from_pos = 0
last = [[3, 6, 7, 11, 12, 17],
        [3, 13, 28, 34, 38, 42],
        [5, 12, 24, 29, 32, 42]]
print(f"finished to set environemnt. ({datetime.now()})")
MAX_MODEL_CNT = 14

finished to set environemnt. (2025-05-16 11:59:13.295492)


In [141]:
# data prepare and set base model
dataset = DataLoader(data_dir=data_dir,
                     training_length=training_length,
                     window_prev=window,
                     mode=mode,
                     from_pos=from_pos
                    )

dataset_dicts = {
    1: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=0
                  ),
    2: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=300
                  )
    }
print(f'completed to load data. {datetime.now()}')

DataLoader.preproc_csv origin = 570, [[ 601    2   16 ...   31   34   35]
 [ 602   13   14 ...   27   30   38]
 [ 603    2   19 ...   26   27   43]
 ...
 [1168    9   21 ...   30   33   37]
 [1169    5   12 ...   26   39   42]
 [1170    3   13 ...   34   38   42]]
rDataLoader.preproc_csv aw_np_proc = 570, [[16 19 31 34 35]
 [14 22 27 30 38]
 [19 25 26 27 43]
 ...
 [21 24 30 33 37]
 [12 24 26 39 42]
 [13 28 34 38 42]]
DataLoader.preproc_csv origin = 570, [[ 601    2   16 ...   31   34   35]
 [ 602   13   14 ...   27   30   38]
 [ 603    2   19 ...   26   27   43]
 ...
 [1168    9   21 ...   30   33   37]
 [1169    5   12 ...   26   39   42]
 [1170    3   13 ...   34   38   42]]
rDataLoader.preproc_csv aw_np_proc = 570, [[16 19 31 34 35]
 [14 22 27 30 38]
 [19 25 26 27 43]
 ...
 [21 24 30 33 37]
 [12 24 26 39 42]
 [13 28 34 38 42]]
DataLoader.preproc_csv origin = 570, [[ 601    2   16 ...   31   34   35]
 [ 602   13   14 ...   27   30   38]
 [ 603    2   19 ...   26   27   43]
 ...
 [116

In [142]:
###########
## layers
## LSTM Neural 계층 선언
###########
"""
Activation (Output):
    linear: No activation, output is directly passed through.
    relu: Rectified Linear Unit, max(x, 0).
    sigmoid: Sigmoid function, output between 0 and 1.
    tanh: Hyperbolic tangent, output between -1 and 1.
    softmax: Normalizes output to a probability distribution.
    elu: Exponential Linear Unit.
    selu: Scaled Exponential Linear Unit.

Recurrent Activation:
    sigmoid: Commonly used for gates in LSTM.
    hard_sigmoid: A faster, less computationally expensive version of sigmoid.
    tanh: Can be used, but sigmoid is more typical for gates.
"""
lstm_layers = {
    1: [LSTM(hid_dim,
             activation=ActivationOutput.elu.name,
             return_sequences=True),
        LSTM(hid_dim,
             return_sequences=False,
             activation=ActivationOutput.elu.name,
             recurrent_activation="hard_sigmoid")],    
    2: [LSTM(hid_dim,
             activation=ActivationOutput.elu.name,
             return_sequences=True),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.elu.name),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.elu.name),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.elu.name),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.elu.name),
        LSTM(hid_dim,
             return_sequences=False,
             activation=ActivationOutput.elu.name,
             recurrent_activation="hard_sigmoid")],
    3: [LSTM(hid_dim,
             activation=ActivationOutput.selu.name,
             return_sequences=True),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.selu.name),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.selu.name),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.selu.name),
        LSTM(hid_dim,
             return_sequences=False,
             activation=ActivationOutput.elu.name,
             recurrent_activation="hard_sigmoid")],
    4: [LSTM(hid_dim,
             activation=ActivationOutput.selu.name,
             return_sequences=True),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.selu.name),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.sigmoid.name),
        LSTM(hid_dim,
             return_sequences=False,
             activation=ActivationOutput.elu.name,
             recurrent_activation="hard_sigmoid")],
    5: [LSTM(hid_dim,
             activation=ActivationOutput.selu.name,
             return_sequences=True),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.selu.name),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.sigmoid.name),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.sigmoid.name),
        LSTM(hid_dim,
             return_sequences=False,
             activation=ActivationOutput.elu.name,
             recurrent_activation="hard_sigmoid")],
    6: [LSTM(hid_dim,
             activation=ActivationOutput.elu.name,
             return_sequences=True),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.elu.name),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.softmax.name),
        LSTM(hid_dim,
             return_sequences=False,
             activation=ActivationOutput.sigmoid.name,
             recurrent_activation="hard_sigmoid")],
    7: [LSTM(hid_dim,
             activation=ActivationOutput.elu.name,
             return_sequences=True),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.elu.name),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.softmax.name),
        LSTM(hid_dim,
             return_sequences=False,
             activation=ActivationOutput.softmax.name,
             recurrent_activation="hard_sigmoid")]
    }

dense_layers = {0: [Dense(45, activation='softmax'),
                    Dense(45, activation='sigmoid')],
                7: [Dense(45, activation='softmax')],
               }
layers = [lstm_layers[1] + dense_layers[0], # 1
          lstm_layers[2] + dense_layers[0], # 2
          lstm_layers[3] + dense_layers[0], # 3
          lstm_layers[4] + dense_layers[0], # 4
          lstm_layers[5] + dense_layers[0], # 5
          lstm_layers[6] + dense_layers[0], # 6
          lstm_layers[7] + dense_layers[7], # 7
          lstm_layers[1] + dense_layers[0], # 8
          lstm_layers[2] + dense_layers[0], # 9
          lstm_layers[3] + dense_layers[0], # 10
          lstm_layers[4] + dense_layers[0], # 11
          lstm_layers[5] + dense_layers[0], # 12
          lstm_layers[6] + dense_layers[0], # 13
          lstm_layers[7] + dense_layers[7], # 14
          ]

datasets = [dataset_dicts[1], # 1
            dataset_dicts[1], # 2
            dataset_dicts[1], # 3
            dataset_dicts[1], # 4
            dataset_dicts[1], # 5
            dataset_dicts[1], # 6
            dataset_dicts[1], # 7
            dataset_dicts[2], # 8
            dataset_dicts[2], # 9
            dataset_dicts[2], # 10
            dataset_dicts[2], # 11
            dataset_dicts[2], # 12
            dataset_dicts[2], # 13
            dataset_dicts[2], # 14
           ]
models = [None for i in range(MAX_MODEL_CNT)]
#matched_cnts = [0 for i in range(MAX_MODEL_CNT)]
#selected_fives = [None for i in range(MAX_MODEL_CNT)]
#matched_list = [None for i in range(MAX_MODEL_CNT)]
#predicted_all = [None for i in range(MAX_MODEL_CNT)]

matched_cnts = {}
selected_fives = {}
matched_list = {}
predicted_all = {}

print(f'completed to set evn for all models. {datetime.now()}')

completed to set evn for all models. 2025-05-16 11:59:20.175582


In [143]:
# all : create model and training
print(f'start to train all models. {datetime.now()}')
for i in range(MAX_MODEL_CNT):
    print(f'{i+1}\'s training. status=start {datetime.now()}')
    model = create_model(id=i,
                         model_type="lstm4",
                         layers=layers[i],
                         dataset=datasets[i],
                         hid_dim=hid_dim,
                         verbose=False
                        )
    models[i] = (model, layers[i])
    print(f'{i+1}\'s training. status=end {datetime.now()}')
print(f'completed to train all models. {datetime.now()}')

start to train all models. 2025-05-16 11:59:34.096475
1's training. status=start 2025-05-16 11:59:34.096721
1's training. status=end 2025-05-16 12:00:22.220433
2's training. status=start 2025-05-16 12:00:22.220531
2's training. status=end 2025-05-16 12:01:54.594569
3's training. status=start 2025-05-16 12:01:54.594669
3's training. status=end 2025-05-16 12:03:21.167617
4's training. status=start 2025-05-16 12:03:21.167748
4's training. status=end 2025-05-16 12:04:27.401504
5's training. status=start 2025-05-16 12:04:27.401606
5's training. status=end 2025-05-16 12:05:45.097638
6's training. status=start 2025-05-16 12:05:45.097764
6's training. status=end 2025-05-16 12:06:50.245797
7's training. status=start 2025-05-16 12:06:50.245895
7's training. status=end 2025-05-16 12:07:51.880674
8's training. status=start 2025-05-16 12:07:51.880797
8's training. status=end 2025-05-16 12:08:18.007635
9's training. status=start 2025-05-16 12:08:18.007755
9's training. status=end 2025-05-16 12:09:05

In [117]:
print(f'check to create model. {datetime.now()}')

check to create model. 2025-05-16 11:40:46.148590


In [6]:
# all : predict #1
greed_prediction = [None for i in range(MAX_MODEL_CNT)]
random_pred = [None for i in range(MAX_MODEL_CNT)]
for i in range(MAX_MODEL_CNT):
    greed_prediction[i] = models[i][0].predict_numbers("greed", trial=1)
    random_pred[i] = models[i][0].predict_randomely(trial=1)
    print(f'{i+1}\'s greed_prediction = {greed_prediction[i]}')
    print(f'{i+1}\'s random_pred = {random_pred[i]}')
print(f'comleted to predict model. {datetime.now()}')

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1's greed_prediction = [array([38, 33, 44, 19, 18, 45])]
1's random_pred = [[24, 2, 44, 11, 14, 8]]
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step
2's greed_prediction = [array([38, 33, 44, 19, 18, 45])]
2's random_pred = [[11, 17, 39, 28, 35, 14]]
comleted to predict model. 2025-05-15 17:10:06.097227


In [136]:
def predict_all_model(model_cnt=MAX_MODEL_CNT):
    matched_cnts = [0 for i in range(MAX_MODEL_CNT)]
    selected_fives = [None for i in range(MAX_MODEL_CNT)]
    matched_list = [None for i in range(MAX_MODEL_CNT)]
    predicted_all = [None for i in range(MAX_MODEL_CNT)]
    for i in range(MAX_MODEL_CNT):
        title = f"history #{i+1}"
        if models[i] is not None:
            models[i][0].verb = "None"
            matched_cnts[i], \
            selected_fives[i], \
            matched_list[i], \
            predicted_all[i] = get_predicted(
                title=title,
                model=models[i][0],
                mode=mode2,
                use_pre=False,
                last=last,
                verbose=False,
                trial=5
                )
    return matched_cnts, selected_fives, matched_list, predicted_all


def repeat_predict_all_model(trial=1, model_cnt=MAX_MODEL_CNT):
    """ repeat_predict_all_model """
    matched_cnts = {}
    selected_fives = {}
    matched_list = {}
    predicted_all = {}
    for t in range(trial):
        matched_cnts[t], \
        selected_fives[t], \
        matched_list[t], \
        predicted_all[t] = predict_all_model(model_cnt)
    return matched_cnts, selected_fives, matched_list, predicted_all


In [137]:
matched_cnts, selected_fives, matched_list, predicted_all = repeat_predict_all_model(trial=20)

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [139]:
print(matched_cnts)
print(matched_list)

{0: [{0: 1, 1: 2, 3: 1, 2: 1}, {0: 2, 1: 2, 2: 1}], 1: [{2: 1, 0: 4}, {0: 3, 1: 1, 2: 1}], 2: [{0: 2, 1: 2, 2: 1}, {0: 3, 1: 1, 2: 1}], 3: [{0: 3, 1: 2}, {1: 1, 0: 4}], 4: [{1: 4, 0: 1}, {2: 2, 1: 2, 0: 1}], 5: [{1: 4, 0: 1}, {0: 2, 1: 2, 2: 1}], 6: [{0: 2, 1: 2, 2: 1}, {0: 4, 1: 1}], 7: [{0: 4, 2: 1}, {0: 2, 3: 2, 1: 1}], 8: [{0: 3, 2: 2}, {1: 2, 0: 3}], 9: [{0: 4, 1: 1}, {0: 2, 1: 2, 2: 1}], 10: [{1: 2, 0: 2, 2: 1}, {2: 1, 1: 2, 0: 2}], 11: [{1: 3, 2: 1, 0: 1}, {0: 3, 2: 1, 1: 1}], 12: [{1: 2, 2: 1, 0: 2}, {1: 3, 2: 2}], 13: [{1: 3, 0: 2}, {2: 1, 0: 2, 1: 2}], 14: [{0: 3, 2: 1, 1: 1}, {1: 4, 2: 1}], 15: [{0: 3, 1: 2}, {0: 2, 1: 3}], 16: [{1: 4, 0: 1}, {0: 1, 2: 2, 1: 2}], 17: [{0: 2, 1: 3}, {1: 1, 0: 4}], 18: [{0: 4, 1: 1}, {1: 2, 0: 1, 2: 1, 3: 1}], 19: [{0: 3, 2: 1, 1: 1}, {1: 3, 0: 2}]}
{0: [[([8, 9, 19, 32, 34, 36], []), ([7, 8, 37, 38, 39, 42], [7]), ([12, 7, 35, 36, 11, 16], [7, 11, 12]), ([3, 12, 14, 22, 30, 41], [3, 12]), ([35, 1, 7, 9, 20, 22], [7])], [([9, 13, 21, 23, 28, 3

In [102]:
import json
import os
from datetime import datetime

def listinlist_2_strinlist(list_in_list):
    """ listinlist_2_strinlist """
    str_in_list = []
    for l in list_in_list:
        str_in_list.append(",".join(l))
    return str_in_list      


def change_matched_info(matched_count, matched_list):
    counts = {}
    cnt = 1
    for c in matched_count:
        counts[cnt] = ",".join([ str(i) for i in c])
        cnt += 1
    cnt = 1
    datas = {}
    for d in matched_list:
        datas[cnt] = d
        cnt += 1
    return counts, datas

def write_json(trial, matched_count, matched_list, append=False):
    suffix = datetime.now().strftime('%y%m%d_%H')
    file_name = f"matched_{suffix}.json"
    matched_dict = {}
    if append:
        if os.path.isfile(file_name):
            with open(file_name, "r") as file:
                matched_dict = json.load(file)
                file.close()
    counts, datas = change_matched_info(matched_count, matched_list)
    if len(counts) > 0 and len(datas) > 0:
        matched_dict[trial] = {}
        matched_dict[trial]["counts"] =  counts
        matched_dict[trial]["datas"] = datas
        print(f'matched_dict = {matched_dict}')
        with open(file_name, "w") as file:
            json.dump(matched_dict, file, indent=4)

In [103]:
# all with trial : print predicted
write_json(trial = 3, matched_count=matched_cnts, matched_list=matched_list, append=True)
print_dict_list(title="matched_cnt", datas=matched_cnts)
for i in range(MAX_MODEL_CNT):
    print_list(title=f"matched_list[{i+1}]", datas=matched_list[i])

matched_dict = {3: {'counts': {1: '0,1', 2: '0,1', 3: '1,2,0', 4: '1,0,2,3', 5: '1,2', 6: '0,2,1', 7: '2,0,1', 8: '0,1,2', 9: '2,1,0', 10: '0,2,1', 11: '0,2', 12: '0,2', 13: '0,1,2', 14: '0,1'}, 'datas': {1: [([4, 8, 19, 23, 33, 37], []), ([9, 14, 28, 31, 33, 45], []), ([6, 18, 26, 36, 41, 45], [6]), ([3, 5, 22, 32, 35, 37], [3]), ([11, 22, 25, 36, 39, 45], [11])], 2: [([18, 22, 26, 27, 38, 45], []), ([28, 16, 11, 40, 5, 39], [11]), ([37, 20, 42, 25, 19, 35], []), ([7, 16, 18, 22, 30, 36], [7]), ([9, 19, 22, 24, 36, 38], [])], 3: [([8, 12, 18, 20, 29, 39], [12]), ([6, 8, 15, 17, 18, 36], [6, 17]), ([33, 4, 39, 25, 12, 34], [12]), ([35, 2, 10, 38, 9, 16], []), ([10, 17, 20, 35, 39, 41], [17])], 4: [([20, 10, 23, 12, 22, 43], [12]), ([5, 8, 9, 15, 32, 37], []), ([1, 8, 11, 26, 32, 35], [11]), ([5, 7, 10, 11, 22, 44], [7, 11]), ([7, 9, 12, 13, 17, 25], [7, 12, 17])], 5: [([7, 19, 34, 38, 41, 44], [7]), ([11, 42, 9, 34, 20, 22], [11]), ([3, 9, 4, 44, 1, 43], [3]), ([3, 7, 9, 10, 33, 41], [

In [234]:
# all (SMALL): predict #2
matched_cnts_small = [0 for i in range(MAX_MODEL_CNT)]
selected_fives_small = [None for i in range(MAX_MODEL_CNT)]
matched_list_small = [None for i in range(MAX_MODEL_CNT)]
predicted_all_small = [None for i in range(MAX_MODEL_CNT)]

for i in range(MAX_MODEL_CNT):
    title = f"history #{i+1}"
    model_num = i
    if models[model_num] is None:
        print(f"model #{model_num+1} is None")
    else:
        models[model_num][0].verb = "None"
        matched_cnts_small[model_num], \
        selected_fives_small[model_num], \
        matched_list_small[model_num], \
        predicted_all_small[model_num] = get_predicted(
            title=title,
            model=models[model_num][0],
            mode=mode2,
            use_pre=False,
            last=last,
            verbose=False,
            trial=5
        )

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


| 회차 | 3이상 ||| 0이 없음 |
| -- | -- | -- | -- | -- |
|  1 | 3, 11 ||| N |
|  2 | 3 ||| N |
|  3 | N ||| N |
|  4 | 12 ||| N |
|  5 | 3, 13 ||| N |
|  6 | 11 ||| N |
|  7 | N ||| N |
|  8 | N ||| N |
|  9 | 3++ ||| N |
| 10 | 7, 12 ||| N |
| 11 | 6, 9, 11 ||| N |
| 12 | 1*, 5, 11, 14 ||| N |
| 13 | N ||| N |
| 14 | 10, 13 ||| N |
| 15 | 5, 7, 8, 13 ||| N |
| 16 | 3 ||| N |
| 17 | 2+, 8, 12 ||| N |
| 18 | 7+ ||| N |
| 19 | 11, 13 ||| N |
| 20 | 4 ||| N |
| 21 | 2, 3 ||| N |
| 22 | 11, 12 ||| N |
| 23 | 1 ||| N |
| 24 | 14 ||| N |
| 25 | 11, 14 ||| 2, 4, 8, 13 |
| 26 | 10+, 14 ||| 3, 6, 11 |
| 27 | 4 ||| 5 |
| 28 | 1, 2, 4, 6+ ||| 5, 13 |
| 29 | _4+_ ||| 3, _4_ |
| 30 | 2+ ||| 13 |
| 31 | 2, 8, 13 ||| 3 |
| 32 | 5, 6 ||| 2, 13 |
| 33 | N ||| N |
| 34 | 2 ||| 6, 12 |
| 35 | 8 ||| N |
| 36 | _5_ ||| _5_ |
| 37 | N ||| 7 |
| 38 | 5, 14+ ||| 2, 7 |
| 39 | 2, 3, 5, 8 ||| 1, 5, 12, 13 |
| 40 | 5, 7, 13 ||| 4, 14 |
| 41 | _10_ ||| _10_ |
| 42 | 13 ||| 14 |
| 43 | 1, 11 ||| 5 |
| 44 | 5, 13 ||| 12 |
| 45 | 6 ||| N |
| 46 | 11 ||| 9 |
| 47 | 5, 14* ||| 5, 6 |
| 48 | N ||| 10, 12 |
| 49 | _7_, 11 ||| _7_ |
| 50 | 8 ||| 2, 9, 12 |
| 51 | 1, 5, 8, 14+ ||| N |
| 52 | N ||| N |
| 53 | 6, 7 ||| 10, 11 |
| 54 | 9 ||| 12 |
| 55 | 14 ||| 7 |
| 56 | 11* ||| 3, 6 |
| 57 | _7_, _12_ ||| _7_, _12_ |
| 58 | N ||| 3 |
| 59 | N ||| 3 |
| 60 | 1, 9, _10_ ||| 4, 5, _10_, 14 |


- {14:6, 13:8, _12:4_, **11:10**, _10:3_, _9:1_, 8:7, 7:5, _6:4_, 5:10, _4:4_, 3:7, 2:7, 1:6
- {14:8, 13:8, _12:5_, **11:11**, _10:4_, _9:3_, 8:7, 7:7, _6:5_, 5:10, _4:4_, 3:7, 2:7, 1:6}
- {**11:11**, 5:10}

In [235]:
# all (SMALL): print predicted
#write_json(trial = 3, matched_count=matched_cnts_small, matched_list=matched_list_small, append=True)
print_dict_list(title="matched_cnt", datas=matched_cnts_small)
#for i in range(MAX_MODEL_CNT):
#    print_list(title=f"matched_list[{i+1}]", datas=matched_list_small[i])

matched_cnt
	1's : {0: 1, 1: 2, 2: 1, 3: 1}
	2's : {0: 1, 1: 3, 2: 1}
	3's : {0: 2, 1: 3}
	4's : {1: 4, 2: 1}
	5's : {1: 5}
	6's : {0: 1, 1: 1, 2: 3}
	7's : {0: 2, 1: 2, 2: 1}
	8's : {0: 4, 2: 1}
	9's : {0: 1, 1: 2, 2: 1, 3: 1}
	10's : {1: 3, 2: 1, 3: 1}
	11's : {0: 3, 1: 2}
	12's : {0: 1, 1: 4}
	13's : {0: 3, 1: 2}
	14's : {1: 3, 2: 2}


In [ ]:
# all : save model
for model in models:
    if model is not None:
        model[0].save()

In [ ]:
# all : model info
#################
# model info
#################
for model in models:
    if model is not None:
        print(f'model:\n\t{model[0].model.summary()}')
        print(f'layer:\n\t{model[1]}')

In [55]:
# all : model test
mode = "back-test"
if mode == 'back-test':
    for model in models:
        if model is not None:
            greed_prediction_number_set = model[0].predict_numbers("greed", trial=1)
            random_pred_set = model[0].predict_randomely(trial=1)
            model[0].evaluate(greed_prediction_number_set)
            print("---------Random baseline-------------")
            model[0].evaluate(random_pred_set)

In [ ]:
for i in range(3):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
for i in range(5):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
# one : create model and training
id = 6
print(f'start to train. [id={id}]')
models[6] = create_model_v1(id=6, dataset=dataset, epoch=10, verbose=True)
print(f'model = {models[6]}')
print(f'end to train. [id={id}]')

In [ ]:
# predict #1
title = "history #1"
model_num = 0 # real 1
if models[model_num] is None:
    print(f"model #{model_num+1} is None")
else:
    models[model_num][0].verb = "None"
    matched_cnts[model_num], \
    selected_fives[model_num], \
    matched_list[model_num], \
    predicted_all[model_num] = get_predicted(
        title=title,
        model=models[model_num][0],
        mode=mode2,
        use_pre=False,
        last=last,
        verbose=True,
        trial=10
        )

In [ ]:
# predict #2
title = "history #2"
model_num = 1 # real 2
if models[model_num] is None:
    print(f"model #{model_num+1} is None")
else:
    models[model_num][0].verb = "None"
    matched_cnts[model_num], \
    selected_fives[model_num], \
    matched_list[model_num], \
    predicted_all[model_num] = get_predicted(
        title=title,
        model=models[model_num][0],
        mode=mode2,
        use_pre=False,
        last=last,
        verbose=True,
        trial=10
        )

In [ ]:
# predict #3
title = "history #3"
model_num = 2 # real 3
if models[model_num] is None:
    print(f"model #{model_num+1} is None")
else:
    models[model_num][0].verb = "None"
    matched_cnts[model_num], \
    selected_fives[model_num], \
    matched_list[model_num], \
    predicted_all[model_num] = get_predicted(
        title=title,
        model=models[model_num][0],
        mode=mode2,
        use_pre=False,
        last=last,
        verbose=True,
        trial=10
        )

In [ ]:
# predict #4
title = "history #4"
model_num = 3 # real 4
if models[model_num] is None:
    print(f"model #{model_num+1} is None")
else:
    models[model_num][0].verb = "None"
    matched_cnts[model_num], \
    selected_fives[model_num], \
    matched_list[model_num], \
    predicted_all[model_num] = get_predicted(
        title=title,
        model=models[model_num][0],
        mode=mode2,
        use_pre=False,
        last=last,
        verbose=True,
        trial=10
        )

In [ ]:
# predict #5
title = "history #5"
model_num = 4 # real 5
if models[model_num] is None:
    print(f"model #{model_num+1} is None")
else:
    models[model_num][0].verb = "None"
    matched_cnts[model_num], \
    selected_fives[model_num], \
    matched_list[model_num], \
    predicted_all[model_num] = get_predicted(
        title=title,
        model=models[model_num][0],
        mode=mode2,
        use_pre=False,
        last=last,
        verbose=True,
        trial=10
        )

In [ ]:
# predict #6
title = "history #6"
model_num = 5 # real 6
if models[model_num] is None:
    print(f"model #{model_num+1} is None")
else:
    models[model_num][0].verb = "None"
    matched_cnts[model_num], \
    selected_fives[model_num], \
    matched_list[model_num], \
    predicted_all[model_num] = get_predicted(
        title=title,
        model=models[model_num][0],
        mode=mode2,
        use_pre=False,
        last=last,
        verbose=True,
        trial=10
        )

In [22]:
# [3, 6, 7, 11, 12, 17]

In [29]:
new1 = flat_data(prediction_number_set22)
new_dict = {}
for n in new1:
    if n in new_dict:
        new_dict[n] += 1
    else:
        new_dict[n] = 1
# print(f'new_dict = {new_dict}')
top, all = get_sorted_n_values(new_dict)
print(f'all = {all}')
print(f'top = {top}')
# print(f'new_dict = {new_dict}')
bottom, all = get_sorted_n_values(new_dict, is_sort=False)
print(f'bottom = {bottom}')

NameError: name 'prediction_number_set22' is not defined

In [ ]:
for i in range(5):
    print(f'{i} : {get_random_in_list(prediction_number_set2, 5)}')

In [ ]:
print_data(title='greed_predicted',
           data_set=greed_prediction_number_set,
           add_val=0, need_sort=True)
print(f'greed_prediction_number_set\n\t\t{greed_prediction_number_set}')

In [ ]:
# when 
"""
trial=20 # help = "how much trials to generate")
training_length=1 # default = 0.9)
epoch=100 # default = 3
batch=3 # default = 1
model='lstm2' # help = "lstm or lstm2")
hid_dim = 128
"""
for i in range(6):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
print(f'model={model}')

In [ ]:
random_pred_set = model2.predict_randomely(trial=1)
print(f'prediction_number_set = {random_pred_set}')

In [ ]:
# 1170 : 3·13·28·34·38·42

last = [
    [6, 8, 37, 40, 41, 44],
    [8, 10, 20, 25, 33, 37],
    [4, 18, 27, 32, 40, 43],
    [2, 8, 10, 31, 33, 35],
    [15, 20, 28, 38, 41, 45],
    [12, 26, 13, 5, 31, 38]
]

In [ ]:
#1171 : [3, 6, 7, 11, 12, 17]